# setup 

Do these steps before you run this notebook!

First steps: setup an environment that includes the pointCollection package

> mamba env create -f wf_pointCollection.yml
> conda activate wf_pointCollection
> python -m ipykernel install --user --name wf_pointCollection

Reload the jupyter window, and choose wf_pointCollection as your notebook environment, then go forward with this notebook.




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sliderule import icesat2
import sliderule
import pointCollection as pc
import pyproj
from scipy import linalg
import datetime
import geopandas as gpd
sliderule.set_verbose(False)
pyproj.datadir.set_data_dir('/home/jovyan/envs/wf_pointCollection/share/proj')
%matplotlib widget

# The workflow:

This is a quick-and-dirty representation of how we would map height-change rates for a small region on the ice sheet.  The workflow could be scaled up to make a map of elevation-change rate for an ice sheet with the caveats that the data volume increases dramatically toward the poles and that fitting a single elevation-change rate does not take full advantage of the data.

Workflow steps:

1. Read all available ATL06 data for a 40x40-km box centered on x0, y0
2. Filter the data for atl06_quality_summary==0
3. Select data in each 1x1 km box in the dataset, fit the selected data with a vertically moving plane

The remaining cells in the notebook plot the results


In [ ]:
%%time
xy0 = [0., -3.0e6]
crs=3413
W=4.e4
pad=np.array([-W/2, W/2])
bbox = [xy0[0] + pad[[0, 1, 1, 0, 0]], xy0[1] + pad[[0, 0, 1, 1, 0]]]
# generate a clipping polygon in a lat/lon coordinate system
bbox_ll = pyproj.Transformer.from_crs(pyproj.CRS(crs), 
                                      pyproj.CRS(4326))\
                    .transform(*bbox)
poly = [{"lat":lat, "lon":lon} for lon, lat in zip(*bbox_ll)]

save_file = f'ATL06_E{int(xy0[0]/1.e3)}_N{int(xy0[1]/1.e3)}.parquet'
icesat2.atl06sp(
   {
    "poly": poly,
    "t0":"2018-10-13T00:00:00Z",
    "t1":"2025-10-13T00:00:00Z", 
   
    "output": {
      "format": "parquet",
      "as_geo": True,
      "path": save_file,
      "with_checksum": False
    }})
df=gpd.read_parquet(save_file).to_crs(3413)

In [ ]:
# use the pointCollection library to get access to tools to iterate over the data

In [ ]:
%%time
# read selected fields out of the dataframe, store them in a pointCollection.data structure
D=pc.data().from_dict({'x':np.array(df.geometry.x),'y':np.array(df.geometry.y),'time':(np.array(df.index) - np.datetime64(datetime.datetime(2018, 1, 1))).astype(float)/1.e9/24/3600/365.25 +2018})
D.assign({field:np.array(df[field]) for field in ['h_li','h_li_sigma', 'atl06_quality_summary']})
# select only points for which atl06_quality_summary == 0
D.index(D.atl06_quality_summary==0)

In [ ]:
%%time
# make a function that fits a time-varying plane to the data for a selected set of points


def dhdt_est(D, ind):
    xx=D.x[ind]
    yy=D.y[ind]
    zz=D.h_li[ind]
    G=np.c_[np.ones(len(ind)), D.time[ind]-2020, xx-np.mean(xx), yy-np.mean(yy)]
    sigma = 1.e4
    keep = np.ones(G.shape[0], dtype=bool)
    keep_last = np.zeros_like(keep)
    n_iterations=0
    try:
        # iterate fit to reduce outliers
        while not np.all(keep_last==keep) and n_iterations  < 5:
            n_iterations += 1
            m=linalg.lstsq(G[keep,:], zz[keep])[0]
            r=zz-G@m
            sigma_r = np.nanstd(r[keep])
            keep = np.abs(r) < 3*np.maximum(sigma, 0.1)
        # estimate errors
        G1 = G[keep,:]
        Ginv = np.linalg.inv(G1.T@G1)@G1.T
        Cm = Ginv@Ginv.T*(np.maximum(0.03**2, sigma_r**2))
        sigma_dhdt = np.sqrt(Cm[1][1])
        return m[1], np.sum(keep), sigma_r, sigma_dhdt
    except np.linalg.LinAlgError:
        return np.nan, np.nan, np.nan, np.nan

# apply the dh/dt function to each 2.5 km square in the data
D_dhdt = pc.apply_bin_fn(D, fn=dhdt_est, res=1.e3, fields=['dhdt','count', 'sigma_r', 'sigma_dhdt'])

In [ ]:
# plot the results:

uxy = pc.unique_by_rows(np.round(np.c_[D.x, D.y]/100)*100)


hf, hax=plt.subplots(1, 3, figsize=[12,4], layout='constrained', sharex=True, sharey=True)
plt.sca(hax[0])
plt.plot(uxy[:,0], uxy[:,1],'k.', markersize=1, zorder=-3)
plt.gca().set_title('coverage')


plt.sca(hax[1])
small_σ= D_dhdt.sigma_dhdt < 0.125
plt.plot(D_dhdt.x[small_σ==0], D_dhdt.y[small_σ==0],'.', color='darkgray')
plt.colorbar(
    plt.scatter(D_dhdt.x[small_σ], D_dhdt.y[small_σ], 3, c=D_dhdt.dhdt[small_σ], cmap='RdBu', clim=[-1, 1]), 
    label='dh/dt, m/yr', shrink=0.5, extend='both')
plt.gca().set_facecolor('lightgray')
plt.gca().set_title('dh/dt')


plt.sca(hax[2])
plt.colorbar(
    plt.scatter(D_dhdt.x, D_dhdt.y, 3, c=D_dhdt.sigma_dhdt, cmap='magma', clim=[0, 1]), 
    label='dh/dt uncertainty, m/yr', shrink=0.5, extend='max')
plt.gca().set_facecolor('lightgray')
plt.gca().set_title('uncertainty')

for ax in hax: 
    ax.set_aspect(1)

# Speed from test run:

For xy0 = [0, -3.e6] in Greenland,
* Fetching data: 28.8 s
* Reading cache: 456 ms
* Processing: 2.26 s
